In [1]:
#spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
data = spark.read.load('/work/data/AGOSTO_2022_PARQUET_FINAL')
#data

In [3]:
KAFKA_BOOTSTRAP_SERVERS = "kafka:9092"
KAFKA_TOPIC = "traffic_sensor"
SCHEMA = data.schema

In [6]:
data.printSchema()

root
 |-- EQP_ID: long (nullable = true)
 |-- DATE_TIME: timestamp (nullable = true)
 |-- MILLISECOND: long (nullable = true)
 |-- CLASSIFICATION: string (nullable = true)
 |-- ROAD_LANE: long (nullable = true)
 |-- ADDRESS_ID: long (nullable = true)
 |-- ROAD_SPEED: string (nullable = true)
 |-- VEHICLE_SPEED: string (nullable = true)
 |-- VEHICLE_LENGTH: string (nullable = true)
 |-- SERIAL_NUMBER: long (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- DIRECTION: string (nullable = true)



In [4]:
df_traffic_stream = spark\
    .readStream.format("kafka")\
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS)\
    .option("subscribe", KAFKA_TOPIC)\
    .option("startingOffsets", "earliest")\
    .load()

In [8]:
import pyspark.sql.functions as F

df_traffic_stream = df_traffic_stream\
    .select(
        F.from_json(
            # decode string as iso-8859-1
            F.decode(F.col("value"), "iso-8859-1"),
            SCHEMA
        ).alias("value")
    )\
    .select("value.*")\
    .select(
        F.col("classification").alias("vehicle_type"),
    )

In [9]:
df_traffic_stream.groupBy("vehicle_type")\
    .count()\
    .writeStream\
    .queryName("vehicle_type_count")\
    .outputMode("complete")\
    .format("memory")\
    .start()

22/11/26 19:07:24 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-06ed80e2-cf57-400f-b4b4-f5e02f48e384. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [10]:
from time import sleep
from IPython.display import display, clear_output

In [11]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [12]:
for x in range(10):
    clear_output(wait=True)
    display(spark.sql("SELECT * FROM vehicle_type_count"))
    sleep(5)

vehicle_type,count
AUTOMÓVEL,80507
INDEFINIDO,2
MOTO,13609
CAMINHÃO / ÔNIBUS,5882
